In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math, os
import glob
import warnings
from sklearn.decomposition import PCA

warnings.filterwarnings("ignore")

In [3]:
walk_fft = pd.read_csv("C:\\Users\\Dell\\Documents\\Wifi-Sensing-HAR\\data\\processed_data\\walk_merged_clean_fft.csv")

In [6]:
csi_rows_raw = []
for one_row in walk_fft["CSI_AMP"]:
    one_row = one_row.strip("[]")
    csi_row_raw = [float(x) for x in one_row.split(" ") if x != '']
    csi_rows_raw.append(csi_row_raw)

# Convert the list of lists to a DataFrame and append it to the appropriate DataFrame based on the file name
csi_df = pd.DataFrame(csi_rows_raw)
csi_df["batch"]=walk_fft["batch"]
csi_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,1314.0,54.0,4.508380e-15,1.852044e-14,3.495221e-14,3.089460e-14,49.193495,71.281133,68.883960,66.483081,...,27.073973,45.177428,43.462628,46.872167,55.362442,2.784304e-15,4.691961e-15,5.538592e-15,4.339866e-15,6.751989e-15
1,292.0,12.0,6.114205e-16,4.619386e-15,5.905309e-15,7.087583e-15,13.416408,12.083046,12.083046,15.620499,...,19.209373,18.681542,19.313208,19.209373,21.470911,6.533455e-16,3.491016e-15,2.653954e-15,1.214560e-15,2.264196e-15
2,438.0,18.0,5.446479e-16,5.361803e-15,9.943283e-15,1.215209e-14,18.027756,26.627054,17.000000,24.083189,...,20.124612,22.803508,18.867962,30.066593,24.758837,1.127870e-15,3.468483e-15,2.923084e-15,5.330439e-16,2.420400e-15
3,584.0,24.0,2.395376e-15,5.439190e-15,1.073827e-14,1.586067e-14,18.248288,14.142136,13.928388,17.000000,...,20.808652,18.384776,21.213203,31.827661,18.601075,1.960122e-15,7.439394e-15,6.950964e-16,3.303782e-15,2.569598e-15
4,876.0,36.0,6.907546e-15,1.306855e-14,1.978383e-14,1.913795e-14,30.000000,14.142136,25.612497,37.202151,...,28.635642,36.124784,49.658836,29.427878,37.336309,3.824489e-15,9.834948e-15,2.611359e-15,2.447644e-15,4.616173e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6985,730.0,30.0,2.135117e-15,8.093404e-15,1.388493e-14,1.839657e-14,22.472205,29.698485,24.186773,34.713110,...,38.897301,27.202941,31.890437,31.622777,30.000000,1.302080e-15,9.205376e-15,4.389027e-15,4.093560e-15,4.233507e-15
6986,1168.0,48.0,2.979043e-15,1.595330e-14,2.325216e-14,2.855519e-14,9.055385,7.071068,20.000000,14.142136,...,17.088007,25.179357,22.360680,13.601470,10.198039,1.229639e-15,4.493848e-15,1.967283e-15,3.296208e-15,3.561555e-15
6987,730.0,30.0,4.651777e-15,9.143219e-15,1.393568e-14,1.554179e-14,49.040799,39.924930,50.447993,44.102154,...,44.384682,45.880279,50.695167,29.681644,44.721359,9.810093e-16,5.016518e-15,2.384924e-15,4.156539e-15,6.431468e-15
6988,730.0,30.0,1.336453e-15,1.081834e-14,1.769712e-14,1.818536e-14,17.691806,10.049876,8.062258,15.000000,...,32.802439,34.928498,24.020824,22.825424,31.953091,1.994310e-15,1.087390e-14,1.617796e-15,4.002860e-15,5.594144e-15


### Scale the data

In [119]:
from sklearn.preprocessing import StandardScaler

def perform_scaling(df):
    scaler = StandardScaler()
    scaler = scaler.fit(df)
    scaled_data = scaler.transform(df)
    return scaled_data

scaled_df = perform_scaling(csi_df.iloc[:,4:60])
walk_scaled_df = pd.DataFrame(scaled_df)
walk_scaled_df["batch"] = walk_fft["batch"]

### Flatten the scaled data

In [120]:
def flatten_dataframe(scaled_Dataframe):
    flattened_data = []
    for batch in scaled_Dataframe["batch"].unique():
        batch_data = scaled_Dataframe[scaled_Dataframe["batch"]==batch].drop(columns = ["batch"]).values.flatten()    
        flattened_data.append(batch_data)
    return flattened_data

walk_flat_data = flatten_dataframe(walk_scaled_df)
np.array(walk_flat_data).shape

(233, 1680)

### PCA of the scaled data

In [121]:
def perform_pca(X, n_components):
    """
    Perform PCA on the data.
    """
    pca = PCA(n_components=n_components)
    pca.fit(X)
    new_sample = pca.transform(X)
    return pca, new_sample

pca_obj, pca_data = perform_pca(walk_flat_data, 5)

In [123]:
pd.DataFrame(pca_data)

,0,1,2,3,4
0,-8.207699,-1.977794,-6.365370,4.355251,-3.725889
1,10.310919,16.338574,-3.117715,-8.384086,9.602418
2,-3.680019,-0.551776,-6.056404,-1.289313,-4.909858
3,-2.290608,3.978300,-1.840788,-1.303283,-4.708618
4,-2.431393,-0.527769,-12.110166,-1.549713,-13.409341
...,...,...,...,...,...
228,1.117209,0.387336,1.190031,10.074535,0.688464
229,7.022570,-5.791048,12.472031,-11.202267,-10.552714
230,-0.884795,-17.244014,-0.133573,9.113863,10.839967
231,-2.392655,4.109981,0.192181,0.052973,-5.684837


In [125]:
## Select and plot raw scaled samples
def raw_sample_plot(raw_sample, activity):
    # raw_sample =  pd.DataFrame(np.vstack(downsampled))
    # print(raw_sample)
    plt.figure(figsize = (14,10))

    ax1 = plt.subplot(611)
    plt.plot(raw_sample.iloc[:,0],'r')
    plt.plot(raw_sample.iloc[:,1],'g')
    plt.plot(raw_sample.iloc[:,2],'b')
    plt.plot(raw_sample.iloc[:,3],'cyan')
    plt.plot(raw_sample.iloc[:,4],'brown')
    plt.plot(raw_sample.iloc[:,5],'lightgreen')
    plt.plot(raw_sample.iloc[:,6],'aqua')
    plt.plot(raw_sample.iloc[:,7],'lime')
    plt.plot(raw_sample.iloc[:,8],'orchid')
    plt.plot(raw_sample.iloc[:,9],'navy')
    plt.plot(raw_sample.iloc[:,10],'royalblue')
    plt.plot(raw_sample.iloc[:,11],'peru')
    plt.plot(raw_sample.iloc[:,12],'tan')
    plt.ylabel("Amplitude")
    ax1.set_title(f"Amplitude of subcarrier 6 to 18 while {activity}")

    ax2 = plt.subplot(612)
    plt.plot(raw_sample.iloc[:,13],'r')
    plt.plot(raw_sample.iloc[:,14],'g')
    plt.plot(raw_sample.iloc[:,15],'b')
    plt.plot(raw_sample.iloc[:,16],'cyan')
    plt.plot(raw_sample.iloc[:,17],'brown')
    plt.plot(raw_sample.iloc[:,18],'lightgreen')
    plt.plot(raw_sample.iloc[:,19],'aqua')
    plt.plot(raw_sample.iloc[:,20],'lime')
    plt.plot(raw_sample.iloc[:,21],'orchid')
    plt.plot(raw_sample.iloc[:,22],'navy')
    plt.plot(raw_sample.iloc[:,23],'royalblue')
    plt.plot(raw_sample.iloc[:,24],'peru')
    plt.plot(raw_sample.iloc[:,25],'tan')
    plt.ylabel("Amplitude")
    ax2.set_title(f"Amplitude of subcarrier 19 to 31 while {activity}")

    ax3 = plt.subplot(613)
    plt.plot(raw_sample.iloc[:,26],'r')
    plt.plot(raw_sample.iloc[:,27],'g')
    plt.plot(raw_sample.iloc[:,28],'b')
    plt.plot(raw_sample.iloc[:,29],'cyan')
    plt.plot(raw_sample.iloc[:,30],'brown')
    plt.plot(raw_sample.iloc[:,31],'lightgreen')
    plt.plot(raw_sample.iloc[:,32],'aqua')
    plt.plot(raw_sample.iloc[:,33],'lime')
    plt.plot(raw_sample.iloc[:,34],'orchid')
    plt.plot(raw_sample.iloc[:,35],'navy')
    plt.plot(raw_sample.iloc[:,36],'royalblue')
    plt.plot(raw_sample.iloc[:,37],'peru')
    plt.plot(raw_sample.iloc[:,38],'tan')
    plt.ylabel("Amplitude")
    ax3.set_title(f"Amplitude of subcarrier 33 to 46 while {activity}")

    ax4 = plt.subplot(614)
    plt.plot(raw_sample.iloc[:,39],'r')
    plt.plot(raw_sample.iloc[:,40],'g')
    plt.plot(raw_sample.iloc[:,41],'b')
    plt.plot(raw_sample.iloc[:,42],'cyan')
    plt.plot(raw_sample.iloc[:,43],'brown')
    plt.plot(raw_sample.iloc[:,44],'lightgreen')
    plt.plot(raw_sample.iloc[:,45],'aqua')
    plt.plot(raw_sample.iloc[:,46],'lime')
    plt.plot(raw_sample.iloc[:,47],'orchid')
    plt.plot(raw_sample.iloc[:,48],'navy')
    plt.plot(raw_sample.iloc[:,49],'royalblue')
    plt.plot(raw_sample.iloc[:,50],'peru')
    plt.plot(raw_sample.iloc[:,51],'tan')
    plt.ylabel("Amplitude")
    plt.xlabel("Time")
    ax4.set_title(f"Amplitude of subcarrier 47 to 58 while {activity}")

    plt.subplots_adjust(hspace=0.6)
    plt.show()